# Best x algorithm example

In this notebook we show with an example how the "best x" algorithm works.

In [1]:
# The whole universe of products
products = {"a": {"revenue": 3.1, "probability": 0.05},
            "b": {"revenue": 1.1, "probability": 0.02},
            "c": {"revenue": 5.1, "probability": 0.005},
            "d": {"revenue": 0.1, "probability": 0.01},
            "e": {"revenue": 0.3, "probability": 0.1},
            "f": {"revenue": 0.3, "probability": 0.08},
            "g": {"revenue": 3.1, "probability": 0.12},
            "h": {"revenue": 0.4, "probability": 0.06},
            "i": {"revenue": 0.4, "probability": 0.05},
            "j": {"revenue": 0.01, "probability": 1},
            "k": {"revenue": 0.01, "probability": 0.99},
            "l": {"revenue": 1.1, "probability": 0.1},
            "m": {"revenue": 1.08, "probability": 0.1},
            "n": {"revenue": 3.2, "probability": 0.04},
            "o": {"revenue": 0.001, "probability": 1.0},
            "p": {"revenue": 0.1, "probability": 0.05}}
# Capacity
M = 10


In [2]:
msg = "Products in the example must have positive revenues"
assert min([p[1]["revenue"] for p in products.items()]) > 0.0, msg


In [3]:
from typing import List, Tuple


In [4]:
def key(item: List) -> Tuple:
    return item[1]["revenue"], item[1]["probability"]


### Lema 1 priority order

Lema 1 shows that, given an assortment of product of length $x$, the optimal is obtained by sorting by revenues and probabilities.

In [5]:
# Lema 1 asserts that for fixed attention spans only the assortment is
# needed and the ranking follows from sorting by revenue and then 
# probability.
for id_, info in sorted(products.items(), key=key, reverse=True):
    print("Id:", id_, "description:", info)


Id: c description: {'revenue': 5.1, 'probability': 0.005}
Id: n description: {'revenue': 3.2, 'probability': 0.04}
Id: g description: {'revenue': 3.1, 'probability': 0.12}
Id: a description: {'revenue': 3.1, 'probability': 0.05}
Id: l description: {'revenue': 1.1, 'probability': 0.1}
Id: b description: {'revenue': 1.1, 'probability': 0.02}
Id: m description: {'revenue': 1.08, 'probability': 0.1}
Id: h description: {'revenue': 0.4, 'probability': 0.06}
Id: i description: {'revenue': 0.4, 'probability': 0.05}
Id: e description: {'revenue': 0.3, 'probability': 0.1}
Id: f description: {'revenue': 0.3, 'probability': 0.08}
Id: p description: {'revenue': 0.1, 'probability': 0.05}
Id: d description: {'revenue': 0.1, 'probability': 0.01}
Id: j description: {'revenue': 0.01, 'probability': 1}
Id: k description: {'revenue': 0.01, 'probability': 0.99}
Id: o description: {'revenue': 0.001, 'probability': 1.0}


### Algorithm 1

The algorithm 1 shows us how to get the minimal lexicographical optimal ranking for each fixed attention span $k$.

In [6]:
from collections import defaultdict
from copy import copy


In [7]:
# Algorithm 1
H = defaultdict(lambda: 0.0)
assort = defaultdict(lambda: [])

for k in range(1, min(len(products), M) + 1):
    for j, product in reversed(list(enumerate(sorted(products.items(), 
                                                     key=key, 
                                                     reverse=True)))):
        default = H[j+1, k]
        alternative = (H[j+1, k-1] 
                       + product[1]["probability"] * (product[1]["revenue"]
                                                      - H[j+1, k-1]))
        if alternative >= default:
            H[j, k] = alternative
            aux_list = copy(assort[j+1, k-1])
            aux_list.append(product)
            assort[j, k] = aux_list
        else:
            H[j, k] = default
            assort[j, k] = assort[j+1, k]


In [8]:
rankings = {k: sorted(assort[0, k], key=key, reverse=True) 
            for k in range(1, min(len(products), M) + 1)}
revenues = {k: H[0, k] for k in range(1, min(len(products), M) + 1)}


In [9]:
revenues[6]


0.8039426598400001

In [10]:
rankings[6]


[('c', {'revenue': 5.1, 'probability': 0.005}),
 ('n', {'revenue': 3.2, 'probability': 0.04}),
 ('g', {'revenue': 3.1, 'probability': 0.12}),
 ('a', {'revenue': 3.1, 'probability': 0.05}),
 ('l', {'revenue': 1.1, 'probability': 0.1}),
 ('m', {'revenue': 1.08, 'probability': 0.1})]

In [11]:
eps = 0.000000000000001
for k in range(1, min(M, len(products)) + 1):
    # Testing that H[0, k] is the expected revenue
    revenue = 0.0
    negative_cumm_prob = 1.0
    for _, product_info in rankings[k]:
        prob = product_info["probability"]
        revenue += negative_cumm_prob * prob * product_info["revenue"]
        negative_cumm_prob *= (1 - prob)


    assert abs(revenue - revenues[k]) < eps, f"{k} failed!"


# Random attention span
As shown in Example 1 from [Revenue Maximization and Learning in Products Ranking](https://arxiv.org/abs/2012.03800) the ranking that maximizes the expected revenue in case the attention span is random is not necessarily the optimal for some fixed attention span.

## Best x algorithm
This algorithm proposes to chose some fixed attention span $x$, which $x$? The **best x**!
Given an optimal $\sigma$ ranking for attention span $x$ the expected revenue should be at least:

$$R(\sigma_{x}, x) Prob(X \geq x) = R(\sigma_{x}, x) G(x).$$

The best $x$ is then:

$$x = \text{arg}\max  R(\sigma_{x}, x) G(x)$$

In [12]:
from scipy.stats import randint
import numpy as np

np.seed = 42


In [13]:
# distribution of attention spans
g = randint(1, M + 1)


In [14]:
maximum_lower_bound = 0.0
best_x = 0
for x, revenue_best_assort in revenues.items():
    revenue_lower_bound = revenue_best_assort * (g.sf(x) + g.pmf(x))
    if revenue_lower_bound > maximum_lower_bound:
        best_x = x
        maximum_lower_bound = revenue_lower_bound
 

In [15]:
best_x


4

In [16]:
rankings[best_x]


[('n', {'revenue': 3.2, 'probability': 0.04}),
 ('g', {'revenue': 3.1, 'probability': 0.12}),
 ('a', {'revenue': 3.1, 'probability': 0.05}),
 ('l', {'revenue': 1.1, 'probability': 0.1})]

### Expected revenue
Since in this example we know the exact distribution of attention spans we can calculate the expected revenue of the "best x" strategy.

In [17]:
expected_revenue = 0.0
negative_cumm_prob = 1.0
for i, product in enumerate(rankings[best_x]):
    prob = product[1]["probability"]
    expected_revenue += (negative_cumm_prob * prob 
                         * product[1]["revenue"]
                         # this works since g can only take integer values
                         * (g.sf(i)))
    negative_cumm_prob *= (1 - prob)


In [18]:
expected_revenue


0.6159603200000001